## 나혜랑 같은 파라미터로 돌리기

- image size=(288,432,3)
- optimizer=rmsprop
- learning rate=2e-5
- loss=binary_crossentropy
- epoch=300
- early_stop = EarlyStopping(patience=10, monitor='val_loss')
- batchSize = 16

In [1]:
# gpu 사용 가능한지 확인
# 사용 가능하다면 device_type에 CPU, GPU 2개가 뜸

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3030417381739935518,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9109728789
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 18443242943275343122
 physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"]

## 1. 라이브러리 임포트

In [3]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from glob import glob
from PIL import Image
# import matplotlib.pyplot as plt
import numpy as np

In [1]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from glob import glob
from PIL import Image
# import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import roc_curve, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight

## 2-1. 모델 레이어 쌓기: VGG-16

In [2]:
input_tensor = Input(shape=(288,432,3), dtype='float32', name='input')
# input_tensor = Input(shape=(244,244,3), dtype='float32', name='input')
 
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Flatten()(x)
x = layers.Dense(4096, kernel_initializer='he_normal')(x)
x = layers.Dense(1024, kernel_initializer='he_normal')(x)
x = layers.Dense(100, kernel_initializer='he_normal')(x)

output_tensor = layers.Dense(1, activation='sigmoid')(x)
 
myvgg16 = Model(input_tensor, output_tensor)
myvgg16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 288, 432, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 288, 432, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 288, 432, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 144, 216, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 144, 216, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 144, 216, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 72, 108, 128)      0     

## 2-2. 모델 레이어 쌓기: VGG-19

In [4]:
input_tensor = Input(shape=(288,432,3), dtype='float32', name='input')
# input_tensor = Input(shape=(244,244,3), dtype='float32', name='input')
 
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.MaxPooling2D((2,2))(x)
 
x = layers.Flatten()(x)
x = layers.Dense(4096, kernel_initializer='he_normal')(x)
x = layers.Dense(1024, kernel_initializer='he_normal')(x)
x = layers.Dense(100, kernel_initializer='he_normal')(x)

output_tensor = layers.Dense(1, activation='sigmoid')(x)
 
myvgg19 = Model(input_tensor, output_tensor)
myvgg19.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 288, 432, 3)]     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 288, 432, 64)      1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 288, 432, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 144, 216, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 144, 216, 128)     73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 144, 216, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 72, 108, 128)      0   

## 3. 모델 컴파일, 학습시키기

In [6]:
BATCHSIZE = 8
EPOCHS = 300

proj_folder="C:/Users/PRLAB/final_project_hny"
train_db='dataset/train'
val_db='dataset/val'
test_db='dataset/test'
train_dir = os.path.join(proj_folder, train_db)
val_dir = os.path.join(proj_folder, val_db)
test_dir = os.path.join(proj_folder, test_db)


# 데이터 generator 생성
train_datagen = ImageDataGenerator(rescale=1./255,
                                  horizontal_flip=True)  # data augmentation
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(288,432),
#                                                     target_size=(244, 244),
                                                    batch_size=BATCHSIZE,
                                                    class_mode='binary',
                                                    interpolation='bilinear',
                                                    shuffle=True,)

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(val_dir,
                                                    target_size=(288, 432),
#                                                     target_size=(244, 244),
                                                    batch_size=BATCHSIZE,
                                                    class_mode='binary',
                                                    interpolation='bilinear',
                                                    shuffle=False,)

early_stopping = EarlyStopping(patience=10, monitor='val_loss')

# 체크포인트 콜백 만들기
checkpoint_path='C:/Users/PRLAB/final_project_hny/final_ver_weights/vgg19_rmsprop/checkpoint-batch-8-epoch-{epoch:02d}-val_loss-{val_loss:.4f}.hdf5'
# checkpoint_path='./final_ver_weights/vgg12/checkpoint-batch-16-epoch-{epoch:02d}-loss-{loss:.4f}.hdf5'
checkpoint=ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', verbose=1, save_weights_only=True)


myvgg19.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])  

print("steps_per_epoch: ", len(train_generator))
print("validation_steps: ", len(valid_generator))
history = myvgg19.fit_generator(train_generator,
                                epochs=EPOCHS,
                                steps_per_epoch=len(train_generator),
                                validation_data=valid_generator,
                                validation_steps=len(valid_generator),
                                verbose=1,
                                callbacks=[early_stopping, checkpoint])    


# 그래프 그리기
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(1, len(acc) + 1)
 
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Accuracy')
plt.legend()
plt.figure()
 
plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Loss')
plt.legend()
 
plt.show()

Found 7421 images belonging to 2 classes.
Found 1838 images belonging to 2 classes.
steps_per_epoch:  928
validation_steps:  230
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 928 steps, validate for 230 steps
Epoch 1/300
  1/928 [..............................] - ETA: 3:25:24WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: 


KeyError: 'val_loss'

## 4. 모델 성능평가

- evaluate A dataset

In [3]:
BATCHSIZE = 8
EPOCHS = 300

checkpoint_path='./final_ver_weights/vgg16_rmsprop/checkpoint-batch-8-epoch-125-val_loss-0.3974.hdf5'
test_dir='./dataset/test'
test_datagen = ImageDataGenerator(rescale=1./255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(288, 432),
    batch_size=BATCHSIZE,
    class_mode='binary',
    interpolation='bilinear',
    shuffle=False)

myvgg16.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc']) 

print("-- Evaluate --")
myvgg16.load_weights(checkpoint_path)# Re-evaluate the model
# loss, acc = myvgg16.evaluate(test_generator, steps=5)
loss, acc = myvgg16.evaluate(test_generator, steps= len(test_generator)) 
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


Found 2317 images belonging to 2 classes.
-- Evaluate --
290/290 [==============================] - 22s 77ms/step - loss: 0.4347 - acc: 0.9642
Restored model, accuracy: 96.42%


- predict A dataset

In [4]:
# model = keras.models.load_model('C:\\Users\\PRLAB\\nahye_finalproject\\result_cuda_2\\dataset2-102-0.1826.hdf5')
# test_datagen = ImageDataGenerator(rescale=1./255.0)
# test_generator = test_datagen.flow_from_directory(
#         'C:\\Users\\PRLAB\\nahye_finalproject\\dataset\\test',
#         target_size=(288, 432),    
#         batch_size=16,
#         class_mode='binary',
#         interpolation='bilinear',
#         shuffle=False)

checkpoint_path='./final_ver_weights/vgg16_rmsprop/checkpoint-batch-8-epoch-125-val_loss-0.3974.hdf5'
test_dir='./dataset/test'

test_datagen = ImageDataGenerator(rescale=1./255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(288, 432),
    batch_size=BATCHSIZE,
    class_mode='binary',
    interpolation='bilinear',
    shuffle=False)
myvgg16.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc']) 



''' evaluating HTER '''
y_true = test_generator.classes
y_score = myvgg16.predict_generator(test_generator, steps = len(test_generator)).ravel()
# Calculate EER threshold
fpr, tpr, threshold = roc_curve(y_true, y_score)
fnr = 1 - tpr
eer_threshold = threshold[np.nanargmin(np.absolute((fnr-fpr)))]
# HTER
y_pred = y_score > eer_threshold
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
labels = test_generator.class_indices
print('pred_fake({})  pred_real({})\nactural_fake({})   {:12d}  {:12d}\nactual_real({})   {:12d}  {:12d}\n'.format(labels['0'], labels['1'], labels['0'], tn, fp, labels['1'], fn, tp))
hter = (fp/(tn+fp) + fn/(fn+tp)) * 0.5
# ROC curve
Accuracy = ((tn+tp) / (tn+fp+fn+tp)) * 100.0
roc_auc = roc_auc_score(y_true, y_score)
plt.figure()
lw = 2
plt.plot(fpr, tpr, 'r--', label='ROC_curve (acc_1st = %0.2f%%)' % Accuracy)
# plt.plot([0, 1], [0, 1], color = 'orange', lw=lw, linestyle='--')
plt.xlim([-0.01, 0.21])
plt.ylim([0.71, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc = "lower right")
plt.savefig('protocol_4_001.png')
plt.figure()
# print('EER: {:.4f}\tHTER: {:.4f}'.format(val_eer, hter))
# print('>> finished')

Found 2317 images belonging to 2 classes.
pred_fake(0)  pred_real(1)
actural_fake(0)           1072            36
actual_real(1)             42          1167



NameError: name 'plt' is not defined

- evaluate B dataset

In [6]:
BATCHSIZE = 8
EPOCHS = 300
checkpoint_path='C:/Users/PRLAB/final_project_hny/final_ver_weights/vgg16_rmsprop/checkpoint-batch-8-epoch-125-val_loss-0.3974.hdf5'

# test_dir2='C:/Users/PRLAB/final_project_hny/act_split_final/test'
test_dir2='C:/Users/PRLAB/nahye_finalproject/stft_spec_2'

test_datagen = ImageDataGenerator(rescale=1./255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir2,
    target_size=(288, 432),
    batch_size=BATCHSIZE,
    class_mode='binary',
    interpolation='bilinear',
    shuffle=False)

myvgg16.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc']) 

print("-- Evaluate --")
myvgg16.load_weights(checkpoint_path)# Re-evaluate the model

# loss,acc = myvgg16.evaluate(test_generator, steps=5)
loss, acc = myvgg16.evaluate(test_generator, steps= len(test_generator))
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Found 3149 images belonging to 2 classes.
-- Evaluate --
394/394 [==============================] - 32s 81ms/step - loss: 2.3261 - acc: 0.5999
Restored model, accuracy: 59.99%


- predict B dataset

In [8]:
# model = keras.models.load_model('C:\\Users\\PRLAB\\nahye_finalproject\\result_cuda_2\\dataset2-102-0.1826.hdf5')
# test_datagen = ImageDataGenerator(rescale=1./255.0)
# test_generator = test_datagen.flow_from_directory(
#         'C:\\Users\\PRLAB\\nahye_finalproject\\dataset\\test',
#         target_size=(288, 432),    
#         batch_size=16,
#         class_mode='binary',
#         interpolation='bilinear',
#         shuffle=False)

checkpoint_path='./final_ver_weights/vgg16_rmsprop/checkpoint-batch-8-epoch-125-val_loss-0.3974.hdf5'
test_dir='C:/Users/PRLAB/nahye_finalproject/stft_spec_2'

test_datagen = ImageDataGenerator(rescale=1./255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(288, 432),
    batch_size=BATCHSIZE,
    class_mode='binary',
    interpolation='bilinear',
    shuffle=False)
myvgg16.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc']) 



''' evaluating HTER '''
y_true = test_generator.classes
y_score = myvgg16.predict_generator(test_generator, steps = len(test_generator)).ravel()
# Calculate EER threshold
fpr, tpr, threshold = roc_curve(y_true, y_score)
fnr = 1 - tpr
eer_threshold = threshold[np.nanargmin(np.absolute((fnr-fpr)))]
# HTER
y_pred = y_score > eer_threshold
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
labels = test_generator.class_indices
print('pred_fake({})  pred_real({})\nactural_fake({})   {:12d}  {:12d}\nactual_real({})   {:12d}  {:12d}\n'.format(labels['0'], labels['1'], labels['0'], tn, fp, labels['1'], fn, tp))
hter = (fp/(tn+fp) + fn/(fn+tp)) * 0.5
# ROC curve
Accuracy = ((tn+tp) / (tn+fp+fn+tp)) * 100.0
roc_auc = roc_auc_score(y_true, y_score)
plt.figure()
lw = 2
plt.plot(fpr, tpr, 'r--', label='ROC_curve (acc_1st = %0.2f%%)' % Accuracy)
# plt.plot([0, 1], [0, 1], color = 'orange', lw=lw, linestyle='--')
plt.xlim([-0.01, 0.21])
plt.ylim([0.71, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc = "lower right")
plt.savefig('protocol_4_001.png')
plt.figure()
# print('EER: {:.4f}\tHTER: {:.4f}'.format(val_eer, hter))
# print('>> finished')

Found 3149 images belonging to 2 classes.
pred_fake(0)  pred_real(1)
actural_fake(0)           1402           965
actual_real(1)            319           463



NameError: name 'plt' is not defined